# Character n-gram models

First, we need some training data.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from ngram import NGramModel

In [4]:
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg
print("Available books:", gutenberg.fileids())

Available books: ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/fredrik/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [12]:
fileids = gutenberg.fileids()[:3]
raw_text = gutenberg.raw(fileids)
raw_text = raw_text.replace('\n', ' ')
data = list(raw_text)

In [16]:
model1 = NGramModel(data, 1)
print(model1)

model2 = NGramModel(data, 2)
print(model2)

model3 = NGramModel(data, 3)
print(model3)

model4 = NGramModel(data, 4)
print(model4)

1-gram model with 81 unique keys
2-gram model with 1542 unique keys
3-gram model with 10212 unique keys
4-gram model with 40030 unique keys


In [20]:
print("unigram:", "".join(model1.predict_sequence(200)))
print()
print("bigram:", "".join(model2.predict_sequence(200)))
print()
print("trigram:", "".join(model3.predict_sequence(200)))
print()
print("quadgram:", "".join(model4.predict_sequence(200)))

unigram: oarsh tdoou hMt fevfenps  ooe geeo newstdbreoouas  lelohh y nhtu gaeh bwubath o ab ocaf"aa ottic e ca Sle hw.tIcpaaein  ensea ocdlprirtse nieapdseeiehralirthc tre 'ei rwnigdae oauetoushfcyrlbaelrvanao

bigram: eved wis spoe veatele ant ho hat, a thonger th whels s reinghe iery cesedepowemos y o " yit, aliethaderse y othilfreeshad  "Wolwaly. livouand Sis otikeasinit fe iten he tished it or d. th in ben sif a

trigram: e ralow ancep eversels," awitabod nor feensuchairself iseetwe thirs.--brive-- Oner I duld Mishime?  He must shaden apted; beed but themands waness. "Wess poing oust or a vell beentme, hisat undeat for

quadgram: peopless Batenting his withought be in told by them.--  "Will; and formater's can alls was, want amuston, but pers the want, I knew that he for in ally said her thank stable tyrand not sheerfectation 
